<a href="https://colab.research.google.com/github/jorgenotte/TiVA/blob/main/TiVA_OCDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar librerías necesarias

import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Crear DataFrame con el mapeo de codigos nace y codigos geo
mapgeo=pd.read_excel('/content/drive/MyDrive/TiVA/mapeo.xlsx','geo').set_index('cod')
mapnace=pd.read_excel('/content/drive/MyDrive/TiVA/mapeo.xlsx','nace').set_index('cod')

# Convertir los DataFrame en diccionarios
mapgeo=dict(mapgeo.to_dict()['geo'])
mapnace=dict(mapnace.to_dict()['nace'])

In [ ]:
# Leer el fichero TiVA en un dataframe [3648 rows × 4109 columns]
ruta='/content/drive/MyDrive/TiVA/TiVA_2020.csv'
df=pd.read_csv(ruta)
df

,V1,ARG_A01_02,ARG_A03,ARG_B05_06,ARG_B07_08,ARG_B09,ARG_C10T12,ARG_C13T15,ARG_C16,ARG_C17_18,...,ZAF_GFCF,ZAF_INVNT,ZAF_DPABR,ROW_HFCE,ROW_NPISH,ROW_GGFC,ROW_GFCF,ROW_INVNT,ROW_DPABR,OUT
0,ARG_A01_02,7093.5057,12.6976,0.8651,2.8701,0.0000,20572.6444,478.1061,117.7472,44.1526,...,0.3456,0.0,0.0230,887.6762,0.0856,7.8340,22.0950,43.4881,5.0583,4.856844e+04
1,ARG_A03,0.0000,65.5517,0.0186,0.0000,0.0000,306.9339,0.0000,0.0000,0.0165,...,0.0000,0.0,0.0021,64.6657,0.0294,0.0711,0.0000,1.4968,0.4840,1.794926e+03
2,ARG_B05_06,8.9094,0.0529,147.4795,11.7133,16.2799,18.5212,1.4532,0.8709,2.1615,...,0.0014,0.0,0.0001,0.3630,0.0001,0.0054,0.2522,0.1786,0.0156,1.292543e+04
3,ARG_B07_08,0.9630,0.0004,32.8632,154.1856,0.0000,196.4411,4.3678,0.0425,13.9015,...,0.0000,0.0,0.0000,43.6990,0.0000,0.0000,0.0419,5.0773,0.0000,4.352430e+03
4,ARG_B09,0.0000,0.0000,2534.4600,95.3887,99.8611,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0781,0.0000,0.0000,2.934979e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3643,CN2_S,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000e+00
3644,CN2_T,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000e+00
3645,TLS,2232.7148,80.0940,345.9149,172.9752,74.1402,1525.1022,101.6188,60.9968,194.0880,...,3162.2347,0.0,35.5072,81962.3991,56.9795,-6998.2598,36478.6151,4242.3221,2918.4099,5.946540e+06
3646,VA,21061.8923,1180.3898,7927.0054,2355.0114,1952.1089,20333.7509,2220.7002,949.1259,2147.2274,...,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000e+00


In [ ]:
# Se cambia la orientación del df de un formato ancho a uno largo
df_des = pd.melt(df, id_vars=['V1'], var_name='Variable', value_name='obs_value')
df_des = df_des.rename(columns={'V1': 'row', 'Variable': 'col'}).sort_values(by=['row', 'col']).reset_index(drop=True)
df_des['obs_value'] = df_des['obs_value'].round(2)

In [ ]:
# Se definen las variables de demanda final
valores = ['HFCE', 'NPISH', 'GGFC', 'GFCF', 'INVNT', 'DPABR']

# Expresión regular que coincide con cualquiera de los caracteres exactos especificados
patron = '|'.join(valores)

In [ ]:
# Se crean los siguientes df: (zy=Consumos intermedios + demanda final) / (tls=Taxes) / (va=Value added) / (x=Output)
df_zy = df_des[((df_des['row'].str.contains('ESP', case=False)) | (df_des['row'].str.contains('FRA', case=False))) & ((df_des['col'].str.contains('ESP', case=False)) | (df_des['col'].str.contains('FRA', case=False)))].reset_index(drop=True)
df_tls = df_des[(df_des['row'] == 'TLS') & ((df_des['col'].str.contains('ESP', case=False)) | (df_des['col'].str.contains('FRA', case=False)))].reset_index(drop=True)
df_va = df_des[(df_des['row'] == 'VA') & ((df_des['col'].str.contains('ESP', case=False)) | (df_des['col'].str.contains('FRA', case=False)))].reset_index(drop=True)
df_x = df_des[(df_des['row'] == 'OUT') & ((df_des['col'].str.contains('ESP', case=False)) | (df_des['col'].str.contains('FRA', case=False)))].reset_index(drop=True)

In [ ]:
# Se crea el df por separado de consumos intermedios y demanda final
df_z = df_zy[~df_zy['col'].str.contains(patron, regex=True)].reset_index(drop=True)
df_y = df_zy[df_zy['col'].str.contains(patron, regex=True)].reset_index(drop=True)

In [ ]:
# Se crea el df por separado de (T-S s/productos)
df_tls_ic = df_tls[~df_tls['col'].str.contains(patron, regex=True)].reset_index(drop=True)
df_tls_fd = df_tls[df_tls['col'].str.contains(patron, regex=True)].reset_index(drop=True)

In [ ]:
# Se crea el df por separado del valor añadido
df_va = df_va[~df_va['col'].str.contains(patron, regex=True)].reset_index(drop=True)

In [ ]:
# Se crea el df por separado de la oferta
df_x = df_x[~df_x['col'].str.contains(patron, regex=True)].reset_index(drop=True)

In [ ]:
# Se divide las cadenas de texto que se encuentran en las columnas 'row' y 'col' en dos partes
df_z[['refArea', 'rowIi']] = df_z['row'].str.split('_', n=1, expand=True)
df_z[['counterpartArea', 'colIi']] = df_z['col'].str.split('_', n=1, expand=True)

df_y[['refArea', 'rowIi']] = df_y['row'].str.split('_', n=1, expand=True)
df_y[['counterpartArea', 'colIi']] = df_y['col'].str.split('_', n=1, expand=True)

df_tls_ic[['counterpartArea', 'colIi']] = df_tls_ic['col'].str.split('_', n=1, expand=True)
df_tls_fd[['counterpartArea', 'colIi']] = df_tls_fd['col'].str.split('_', n=1, expand=True)

df_va[['counterpartArea', 'colIi']] = df_va['col'].str.split('_', n=1, expand=True)

df_x[['counterpartArea', 'colIi']] = df_x['col'].str.split('_', n=1, expand=True)

In [ ]:
# Se crea la columna agregar
df_z['agrega'] = df_z['refArea'].map(mapgeo) + '_' + df_z['rowIi'].map(mapnace) + '*' + df_z['counterpartArea'].map(mapgeo) + '_' + df_z['colIi'].map(mapnace)
df_z_colapsado=pd.DataFrame(df_z.groupby(['agrega'])['obs_value'].sum()).reset_index()

df_y['agrega'] = df_y['refArea'].map(mapgeo) + '_' + df_y['rowIi'].map(mapnace) + '*' + df_y['counterpartArea'].map(mapgeo) + '_' + df_y['colIi'].map(mapnace)
df_y_colapsado=pd.DataFrame(df_y.groupby(['agrega'])['obs_value'].sum()).reset_index()

df_tls_ic['agrega'] = df_tls_ic['row'] + '*' + df_tls_ic['counterpartArea'].map(mapgeo) + '_' + df_tls_ic['colIi'].map(mapnace)
df_tls_ic_colapsado=pd.DataFrame(df_tls_ic.groupby(['agrega'])['obs_value'].sum()).reset_index()

df_tls_fd['agrega'] = df_tls_fd['row'] + '*' + df_tls_fd['counterpartArea'].map(mapgeo) + '_' + df_tls_fd['colIi'].map(mapnace)
df_tls_fd_colapsado=pd.DataFrame(df_tls_fd.groupby(['agrega'])['obs_value'].sum()).reset_index()

df_va['agrega'] = df_va['row'] + '*' + df_va['counterpartArea'].map(mapgeo) + '_' + df_va['colIi'].map(mapnace)
df_va_colapsado=pd.DataFrame(df_va.groupby(['agrega'])['obs_value'].sum()).reset_index()

df_x['agrega'] = df_x['row'] + '*' + df_x['counterpartArea'].map(mapgeo) + '_' + df_x['colIi'].map(mapnace)
df_x_colapsado=pd.DataFrame(df_x.groupby(['agrega'])['obs_value'].sum()).reset_index()

In [ ]:
df_z_colapsado['origen']=df_z_colapsado['agrega'].str.split('*').str[0]
df_z_colapsado['destino']=df_z_colapsado['agrega'].str.split('*').str[1]

df_y_colapsado['origen']=df_y_colapsado['agrega'].str.split('*').str[0]
df_y_colapsado['destino']=df_y_colapsado['agrega'].str.split('*').str[1]

df_tls_ic_colapsado['origen']=df_tls_ic_colapsado['agrega'].str.split('*').str[0]
df_tls_ic_colapsado['destino']=df_tls_ic_colapsado['agrega'].str.split('*').str[1]

df_tls_fd_colapsado['origen']=df_tls_fd_colapsado['agrega'].str.split('*').str[0]
df_tls_fd_colapsado['destino']=df_tls_fd_colapsado['agrega'].str.split('*').str[1]

df_va_colapsado['origen']=df_va_colapsado['agrega'].str.split('*').str[0]
df_va_colapsado['destino']=df_va_colapsado['agrega'].str.split('*').str[1]

df_x_colapsado['origen']=df_x_colapsado['agrega'].str.split('*').str[0]
df_x_colapsado['destino']=df_x_colapsado['agrega'].str.split('*').str[1]

In [ ]:
df_z_tabla = df_z_colapsado.pivot_table(index="origen", columns="destino", values="obs_value")

df_y_tabla = df_y_colapsado.pivot_table(index="origen", columns="destino", values="obs_value")

df_tls_ic_tabla = df_tls_ic_colapsado.pivot_table(index="origen", columns="destino", values="obs_value")

df_tls_fd_tabla = df_tls_fd_colapsado.pivot_table(index="origen", columns="destino", values="obs_value")

df_va_tabla = df_va_colapsado.pivot_table(index="origen", columns="destino", values="obs_value")

df_x_tabla = df_x_colapsado.pivot_table(index="origen", columns="destino", values="obs_value")

In [ ]:
matriz_z = df_z_tabla.values
matriz_y = df_y_tabla.values
matriz_tls_ic = df_tls_ic_tabla.values
matriz_tls_fd = df_tls_fd_tabla.values
matriz_va = df_va_tabla.values
matriz_x = df_x_tabla.values

In [ ]:
# Value added to output ratio (VA/X)
ratio = matriz_va / matriz_x

df_ratio = pd.DataFrame(ratio, columns=df_va_tabla.columns, index=df_va_tabla.index)
nuevo_indice = ['VA/X']
df_ratio.index = pd.Index(nuevo_indice, name='origen')

In [ ]:
# Input coefficients (Z/X)
input_coeff = matriz_z / matriz_x

df_input_coeff = pd.DataFrame(input_coeff, columns=df_z_tabla.columns, index=df_z_tabla.index)

In [ ]:
# Matriz identidad
matriz_identidad = np.eye(40)
df_matriz_identidad = pd.DataFrame(matriz_identidad, columns=df_z_tabla.columns, index=df_z_tabla.index)

# Diferencia entre matriz identidad y matriz de coeficientes técnicos
diferencia = matriz_identidad - input_coeff
df_diferencia = pd.DataFrame(diferencia, columns=df_z_tabla.columns, index=df_z_tabla.index)

# Matriz inversa de Leontief
inversa = np.linalg.inv(diferencia)
df_inversa = pd.DataFrame(inversa, columns=df_z_tabla.columns, index=df_z_tabla.index)

In [ ]:
# Gross intermediate trade flows (GRTR_INT)

# Se divide las cadenas de texto que se encuentran en las columnas 'origen' y 'destino' en dos partes
df_z_colapsado[['origen2','x1']] = df_z_colapsado['origen'].str.split('_', n=1, expand=True)
df_z_colapsado[['destino2','x2']] = df_z_colapsado['destino'].str.split('_', n=1, expand=True)
df_z_colapsado = df_z_colapsado.drop(['x1', 'x2'], axis=1)

df_z_grtr = df_z_colapsado.copy()
df_z_grtr['obs_value2'] = np.where(df_z_grtr['origen2'] == df_z_grtr['destino2'], 0, df_z_grtr['obs_value'])
df_z_grtr['agrega2'] = df_z_grtr['origen'] + '*' + df_z_grtr['destino2']

df_z_grtr2=pd.DataFrame(df_z_grtr.groupby(['agrega2'])['obs_value2'].sum()).reset_index()
df_z_grtr2[['origen2','destino2']] = df_z_grtr2['agrega2'].str.split('*', n=1, expand=True)


In [ ]:
df_z_grtr_table = df_z_grtr2.pivot_table(index="origen2", columns="destino2", values="obs_value2")

In [ ]:
# Gross final trade flows (GRTR_FNL)

# Se divide las cadenas de texto que se encuentran en las columnas 'origen' y 'destino' en dos partes
df_y_colapsado[['origen2','x1']] = df_y_colapsado['origen'].str.split('_', n=1, expand=True)
df_y_colapsado[['destino2','x2']] = df_y_colapsado['destino'].str.split('_', n=1, expand=True)
df_y_colapsado = df_y_colapsado.drop(['x1', 'x2'], axis=1)

df_y_grtr = df_y_colapsado.copy()
df_y_grtr['obs_value2'] = np.where(df_y_grtr['origen2'] == df_y_grtr['destino2'], 0, df_y_grtr['obs_value'])
df_y_grtr['agrega2'] = df_y_grtr['origen'] + '*' + df_y_grtr['destino2']

df_y_grtr2=pd.DataFrame(df_y_grtr.groupby(['agrega2'])['obs_value2'].sum()).reset_index()
df_y_grtr2[['origen2','destino2']] = df_y_grtr2['agrega2'].str.split('*', n=1, expand=True)

In [ ]:
df_y_grtr_table = df_y_grtr2.pivot_table(index="origen2", columns="destino2", values="obs_value2")

In [ ]:
ruta_excel = '/content/drive/MyDrive/TiVA/Indicadores TiVA ESP-FRA.xlsx'

with pd.ExcelWriter(ruta_excel, engine='openpyxl') as writer:
    # Guardar cada DataFrame en una hoja diferente
    df_z_tabla.to_excel(writer, sheet_name='Matriz CI', index=True)
    df_y_tabla.to_excel(writer, sheet_name='Matriz DF', index=True)
    df_tls_ic_tabla.to_excel(writer, sheet_name='Impuestos CI', index=True)
    df_tls_fd_tabla.to_excel(writer, sheet_name='Impuestos DF', index=True)
    df_va_tabla.to_excel(writer, sheet_name='VA', index=True)
    df_x_tabla.to_excel(writer, sheet_name='Producción', index=True)
    df_ratio.to_excel(writer, sheet_name='Ratio VA to X', index=True)
    df_z_grtr_table.to_excel(writer, sheet_name='GITF', index=True)
    df_y_grtr_table.to_excel(writer, sheet_name='GFTF', index=True)
    df_input_coeff.to_excel(writer, sheet_name='Input coefficient', index=True)
    df_matriz_identidad.to_excel(writer, sheet_name='Identity matrix', index=True)
    df_diferencia.to_excel(writer, sheet_name='I-A', index=True)
    df_inversa.to_excel(writer, sheet_name='Leontief inverse', index=True)

### **STRUCTURAL INDICATORS - BASED ON THE VALUES IN ICIO TABLES**

In [ ]:
# 4.1 Production (Gross output)

production = round(df_x.loc[df_x['counterpartArea'] == 'ESP', 'obs_value'].sum(), 1)
print("La producción de ESP es:", production)

La producción de ESP es: 2320651.2


In [ ]:
# 4.2 Value added

value_added = round(df_va.loc[df_va['counterpartArea'] == 'ESP', 'obs_value'].sum(), 1)
print("El valor añadido de ESP es:", value_added)

El valor añadido de ESP es: 1169754.4


In [ ]:
# 4.3 Value added as a % of production

df_ratio

destino,ESP_A,ESP_B,ESP_C,ESP_D,ESP_E,ESP_F,ESP_G,ESP_H,ESP_I,ESP_J,...,FRA_K,FRA_L,FRA_M,FRA_N,FRA_O,FRA_P,FRA_Q,FRA_R,FRA_S,FRA_T
origen,,,,,,,,,,,,,,,,,,,,,
VA/X,0.526056,0.40062,0.265738,0.410396,0.401466,0.411112,0.55172,0.382612,0.533517,0.442724,...,0.341608,0.808815,0.466627,0.575436,0.72111,0.82265,0.724136,0.588457,0.608621,1.0


In [ ]:
# 4.4 Gross exports

# Intermediate products
int_exports = round(df_z_grtr.loc[(df_z_grtr['origen2'] == 'ESP') & (df_z_grtr['destino2'] == 'FRA'), 'obs_value2'].sum(), 1)
print("Las exportaciones de productos intermedios para ESP son:", int_exports)

# Final products
final_exports = round(df_y_grtr.loc[(df_y_grtr['origen2'] == 'ESP') & (df_y_grtr['destino2'] == 'FRA'), 'obs_value2'].sum(), 1)
print("Las exportaciones de productos terminados para ESP son:", final_exports)

# Total gross exports
total_exports = round(int_exports + final_exports, 1)
print("Las exportaciones totales para ESP son:", total_exports)

Las exportaciones de productos intermedios para ESP son: 27700.2
Las exportaciones de productos terminados para ESP son: 24362.2
Las exportaciones totales para ESP son: 52062.4


In [ ]:
# 4.5 Gross imports

# Intermediate products
int_imports = round(df_z_grtr.loc[(df_z_grtr['origen2'] == 'FRA') & (df_z_grtr['destino2'] == 'ESP'), 'obs_value2'].sum(), 1)
print("Las importaciones de productos intermedios para ESP son:", int_imports)

# Final products
final_imports = round(df_y_grtr.loc[(df_y_grtr['origen2'] == 'FRA') & (df_y_grtr['destino2'] == 'ESP'), 'obs_value2'].sum(), 1)
print("Las importaciones de productos terminados para ESP son:", final_imports)

# Total gross imports
total_imports = round(int_imports + final_imports, 1)
print("Las importaciones totales para ESP son:", total_imports)

Las importaciones de productos intermedios para ESP son: 21092.3
Las importaciones de productos terminados para ESP son: 13501.2
Las importaciones totales para ESP son: 34593.5


In [ ]:
# 4.6 Gross trade balance

# Difference between Gross Exports and Gross Imports
gross_trade_balance = round(total_exports - total_imports, 1)
print("El balance de exportaciones/importaciones de ESP son:", gross_trade_balance)

El balance de exportaciones/importaciones de ESP son: 17468.9


In [ ]:
# 4.7 Gross exports, partner shares (%)

In [ ]:
# 4.8 Gross imports, partner shares (%)

In [ ]:
# 5.1 Domestic value added content of gross exports